# Tutorial 6: Functions and Scope

## PHYS 2600, Fall 2018

## T6.X - A deep dive with variables and scope

_(Special note: this first problem is a __worked example__, which we'll go through together in class.  You are encouraged to fill this in as you follow along, but you won't be graded on whether you've completed it or not.)_

### Part A

Let's dig into a more complicated example of a function definition, and see what's happening with variables and their scope, line by line.  We'll make the function itself really simple: it will be called `triple(x)`, and will return 3 times whatever `x` is.  

Consider the following code, but __do not run it yet:__

In [ ]:
x=2
y=3

def triple(x):
    print("Inside triple, x is:", x, "and y is:", y)
    z=3*x
    return z

print("Triple sum:", triple(3) + triple(4))
print("At the end, x is:", x)

Now, let's make sure we understand how variables and scope work by __predicting what will happen in the code, step by step.__  In class, I'll do this (with your help!) at the blackboard.  If you're working this exercise at home, make a blank Markdown cell or use a sheet of paper to predict what the code will do.  

For context, here's an example sketch of what the global and local namespaces look like after we call `triple(4)`:

<img src="https://physicscourses.colorado.edu/phys2600/phys2600_sp19/img/scopes.png" width=500px />

When you're finished with your prediction, run the cell and see if the output is what you expected.  Then __check your prediction__ by running the code above through the [Python Tutor](http://www.pythontutor.com/visualize.html#mode=edit).

### Part B

As the diagram hints, we can access global scope from inside a function as well, as long as we don't override the variable name.  __Predict the output of the following example__, and then run it:

In [ ]:
n=2

def global_pow(x):
    return x**n

print(global_pow(3))

n=3
print(global_pow(3))

__Why does changing `n` work like this__, if we do it _after_ we've defined the `global_pow` function already?!  (The answer has to do with _when_ the statement `return x**n` is actually evaluated; again, try the Python Tutor if you're stuck.)

__ANSWER: There is no `n` in the local namespace of `global_pow`, so when we call it, it uses the _current_ value of `n` from the _global_ namespace - hence, its behavior changes when we change `n`.__

You should keep in mind that using global variables in this way is almost always a bad idea!  There are two major reasons:

1. The code is much less clear; to know what `global_pow()` will actually do, you have to go backwards to find where `n` was defined last.
2. If $n$ is used somewhere else in your code, the behavior of `global_pow()` can suddenly and unexpectedly change!

### Part C

Since using global variables has those drawbacks, let's implement a better `pow` function using keyword arguments - which will reveal another surprising behavior!  Again, __predict the output of the following code__, then run it:

In [ ]:
n=2

def keyword_pow(x, n=n):
    return x**n

print(keyword_pow(3))
print(keyword_pow(3,n=5))

n=3
print(keyword_pow(3))
print(keyword_pow(3, n=n))

Once again, if you get stuck the Python Tutor will help.  But the key point to understanding the behavior of the above code is this: although everything _inside_ a function is stored and not executed until later, the function header itself is __evaluated at the moment that we define it__.

The other weird thing about the code above is the appearance of the bizarre-looking statement `n=n`.  Normally, this would do nothing at all!  But when we use it with a function definition or keyword argument, it really means 

`(local) n = (global) n`.  

(You can think of the statement `n=n` as "handing off" the value of `n` from global to local scope.)

## T6.1 - Making your own functions

Let's start with some basic practice making our own functions.  Each of the code examples below contains a function call for a function that hasn't been implemented yet.  In each part below, __write the function in the first cell__ so that the code in the second cell works correctly.

### Part A

We want a function called `hello` that takes no arguments, and returns the string `Hello, world!` when it is called.

In [1]:
# Write your function here

### BEGIN SOLUTION
def hello():
    return "Hello, world!"
    
### END SOLUTION

In [2]:
## Read-only code: should run after you implement the function above.

print(hello())

# Note: this should print only "Hello, world!" - if you see "None" printed out as well, your function is incorrect!
# Remember that you should _return_ the string from your function.

Hello, world!


### Part B

We want a function called `dist_origin` that takes two numbers `x` and `y`, and computes the distance from the origin $(0,0)$ to the point $(x,y)$.  (You'll need to import a square-root function from somewhere!)

In [3]:
# Write your function here

### BEGIN SOLUTION
import numpy as np

def dist_origin(x,y):
    return np.sqrt(x**2 + y**2) 
### END SOLUTION


In [4]:
## Read-only code: should run after you implement the function above.

print(dist_origin(3,4))  # should print 5.0
print(dist_origin(1,-1)) # should print about 1.41

5.0
1.4142135623730951


### Part C

We want a function called `sum_power` that takes a __positional argument__ `N` and a __keyword argument__ `p` (with default value 1), and then computes the sum

$$
S(N,p) \equiv \sum_{k=0}^{N} k^p
$$

_(Hint: use `np.arange` to make an array of numbers, raise them to the appropriate power, and then use `np.sum` to add the array up.)_

In [5]:
# Write your function here

### BEGIN SOLUTION
def sum_power(N,p=1):
    k = np.arange(0,N+1)  # Don't forget to include N at the top end!
    pow_array = k**p
    return np.sum(pow_array)
### END SOLUTION


In [6]:
## Read-only code: should run after you implement the function above.

print(sum_power(10))      # With p=1, should print 55
print(sum_power(10, p=2)) # Should print 385
print(sum_power(2, p=10)) # Should print 1 + 2^10 = 1025

55
385
1025


## T6.2 - Mixing scope with the `global` keyword

There's one question I never addressed about scope in the materials so far: can you _write_ to global variables from inside of a local scope?  The answer is sort of, but it leads to one of the most complicated parts of Python's scoping rules.  For you information, we'll go through that case now.

This section comes with a caveat: _you should almost never use the `global` keyword!_  Just as is the case with the global variable example we saw in lecture, modification of global variables breaks the principle of __encapsulation__, that a function should just be concerned with things locally available to it.  Encapsulation makes it much easier to diagnose errors; if your code depends on global scope, you have to look _everywhere_ for the problem!

### Part A


Although the variables in global namespace are visible within the scope of a function, Python actually distinguishes between _reading_ and _writing_ such variables.  Consider the following example code:

In [7]:
n=2

def adder():
    global n   # ANSWER: this line declares we're going to write to "n" in global namespace.
    n += 1

adder()
print(n)

3


First, read the code and make sure you understand what it's supposed to do, and what the expected outcome of the final `print(n)` is.  Then, run the code and notice that you get an error message, specifically an `UnboundLocalError`.  This is because Python's default behavior is to _not_ allow writing to global variables inside of local scope.

However, we can overrule this behavior by using the `global` keyword.  The statement `global x` tells Python that we want to be able to make full use of the global-namespace variable `x` in our local scope.  __Use the `global` keyword to fix the code above__, so that `adder()` has the expected outcome.

### Part B

One more example of using global scope.  The _Fibonacci sequence_ is defined by the formula
\\[
F_n = F_{n-1} + F_{n-2}
\\]
along with the definitions $F_0 = F_1 = 1$.  Let's make use of global scope to compute numbers from this sequence.  In the cell below, implement a function `iterate_fib()` which uses the values of the global variables `Fn_minus_one` and `Fn_minus_two` to compute the next value in the sequence.  

`iterate_fib()` should __return__ the value $F_n$, and then change `Fn_minus_one` and `Fn_minus_two` to prepare for the next time the function is called.  Remember that the `global` keyword is needed if you're going to assign to variables outside the local scope in a function.

In [8]:
Fn_minus_one = 1
Fn_minus_two = 1

def iterate_fib():

    ### BEGIN SOLUTION
    global Fn_minus_two 
    global Fn_minus_one 

    Fn = Fn_minus_one + Fn_minus_two
    Fn_minus_two = Fn_minus_one
    Fn_minus_one = Fn
    
    return Fn
    ### END SOLUTION
    

In [9]:
## Testing cell; should run without error if your implementation above works!

Fn_minus_one = 1
Fn_minus_two = 1

assert (iterate_fib() == 2)
assert (iterate_fib() == 3)
assert (iterate_fib() == 5)
assert (iterate_fib() == 8)
assert (iterate_fib() == 13)